In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv('/kaggle/input/training-data/Training Data.csv')

In [ ]:
df.drop('Id',axis=1,inplace=True)
df.drop('CITY',axis=1,inplace=True)
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
df['Profession'].value_counts()

In [ ]:
categorical_features=[feature for feature in df.columns if df[feature].dtype=='O']
df[categorical_features].head()

In [ ]:
df['STATE'].replace('Uttar_Pradesh[5]','Uttar_Pradesh',inplace=True)

In [ ]:
df['STATE'].replace('Uttar_Pradesh','UP',inplace=True)
df['STATE'].replace('Madhya_Pradesh','MP',inplace=True)
df['STATE'].replace('West_Bengal','WBengal',inplace=True)
df['STATE'].replace('Jammu_and_Kashmir','J&K',inplace=True)
df['STATE'].replace('Himachal_Pradesh','HP',inplace=True)
df['STATE'].replace('Andhra_Pradesh','AP',inplace=True)

In [ ]:
df['STATE'].unique()

In [ ]:
plt.figure(figsize=(10,7))
sns.countplot(df['House_Ownership'])

In [ ]:
plt.figure(figsize=(10,7))
sns.countplot(df['Car_Ownership'])

In [ ]:
plt.figure(figsize=(10,7))
sns.countplot(df['Married/Single'])

In [ ]:
plt.figure(figsize=(25,7))
sns.countplot(df['STATE'])

In [ ]:
plt.figure(figsize=(10,7))
sns.countplot(df['Risk_Flag'])
## so we need to resample the data for value=0 since the difference is huge.

In [ ]:

from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for feature in categorical_features:
    df[feature]=le.fit_transform(df[feature])
    
df.head()

In [ ]:
x=df.drop('Risk_Flag',axis=1)
y=df['Risk_Flag']
import statsmodels.api as sm
x=sm.add_constant(x)
model=sm.OLS(y,x).fit()
model.summary()
# since the p values are less than 0.05 we can assume no multicollinearity, we can confirm that by corr data too.

In [ ]:
x.corr()

In [ ]:
from sklearn.model_selection import train_test_split
xt,xtest,yt,ytest=train_test_split(x,y,test_size=0.33,random_state=0)

In [ ]:
df_u=pd.concat([xt,yt],axis=1)

In [ ]:
from sklearn.utils import resample
val_0=df_u[df_u.Risk_Flag==0]
val_1=df_u[df_u.Risk_Flag==1]
downsample_0=resample(val_0,replace=True,n_samples=len(val_1),random_state=27)
ds=pd.concat([downsample_0,val_1])
ds.Risk_Flag.value_counts()

In [ ]:
xtrain=ds.drop('Risk_Flag',axis=1)
ytrain=ds['Risk_Flag']

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x = scaler.fit_transform(x)
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=10,criterion='entropy',random_state=0)
rf.fit(xtrain,ytrain)
ypred=rf.predict(xtest)

In [ ]:
from sklearn.model_selection import cross_val_score
cv=cross_val_score(estimator=rf,X=xtrain,y=ytrain,cv=10)
print(abs(cv.mean()))

In [ ]:
from sklearn.metrics import accuracy_score
acc=accuracy_score(ytest,ypred)
acc

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(ytest,ypred)
plt.figure(figsize=(10,7))
plt.title('Confusion matrix for RandomForest')
df_cm=pd.DataFrame(cm,index=[i for i in [0,1]],columns=[i for i in ['predicted 0','predicted 1']])
sns.heatmap(df_cm,annot=True,fmt='g');

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(leaf_size= 2, n_neighbors= 2, p= 1)
knn.fit(xtrain,ytrain)
ypred1=knn.predict(xtest)

In [ ]:
from sklearn.model_selection import cross_val_score
cv=cross_val_score(estimator=knn,X=xtrain,y=ytrain,cv=10)
print(abs(cv.mean()))

In [ ]:
from sklearn.metrics import accuracy_score
acc1=accuracy_score(ytest,ypred1)
acc1

In [ ]:
from sklearn.metrics import confusion_matrix
cm1=confusion_matrix(ytest,ypred1)
plt.figure(figsize=(10,7))
plt.title('Confusion matrix for KNeighbors')
df_cm=pd.DataFrame(cm1,index=[i for i in [0,1]],columns=[i for i in ['predicted 0','predicted 1']])
sns.heatmap(df_cm,annot=True,fmt='g');

In [ ]:
from sklearn.naive_bayes import GaussianNB
gb=GaussianNB()
gb.fit(xtrain,ytrain)
ypred2=gb.predict(xtest)

In [ ]:
from sklearn.model_selection import cross_val_score
cv=cross_val_score(estimator=gb,X=xtrain,y=ytrain,cv=10)
print(abs(cv.mean()))

In [ ]:
from sklearn.metrics import accuracy_score
acc2=accuracy_score(ytest,ypred2)
acc2

In [ ]:
cm2=confusion_matrix(ytest,ypred2)
plt.figure(figsize=(10,7))
plt.title('Confusion matrix for Naive_Bayes')
df_cm=pd.DataFrame(cm2,index=[i for i in [0,1]],columns=[i for i in ['predicted 0','predicted 1']])
sns.heatmap(df_cm,annot=True,fmt='g');

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(criterion='entropy',random_state=0)
dt.fit(xtrain,ytrain)
ypred3=dt.predict(xtest)

In [ ]:
cv=cross_val_score(estimator=dt,X=xtrain,y=ytrain,cv=10)
print(abs(cv.mean()))

In [ ]:
acc3=accuracy_score(ytest,ypred3)
acc3

In [ ]:
cm3=confusion_matrix(ytest,ypred3)
plt.figure(figsize=(10,7))
plt.title('Confusion matrix for DecisionTree')
df_cm=pd.DataFrame(cm3,index=[i for i in [0,1]],columns=[i for i in ['predicted 0','predicted 1']])
sns.heatmap(df_cm,annot=True,fmt='g');

In [ ]:
from sklearn.linear_model import LogisticRegression
lrc=LogisticRegression(random_state=0)
lrc.fit(xtrain,ytrain)
ypred4=lrc.predict(xtest)

In [ ]:
cv=cross_val_score(estimator=lrc,X=xtrain,y=ytrain,cv=10)
print(abs(cv.mean()))

In [ ]:
acc4=accuracy_score(ytest,ypred4)
acc4

In [ ]:
cm4=confusion_matrix(ytest,ypred4)
plt.figure(figsize=(10,7))
plt.title('Confusion matrix for Logistic')
df_cm=pd.DataFrame(cm4,index=[i for i in [0,1]],columns=[i for i in ['predicted 0','predicted 1']])
sns.heatmap(df_cm,annot=True,fmt='g');

In [ ]:
from xgboost import XGBClassifier
xg=XGBClassifier(learning_rate=0.1, 
                    n_estimators=1000, 
                    use_label_encoder=False,
                    random_state=42)
xg.fit(xtrain,ytrain)
ypred5=xg.predict(xtest)

In [ ]:
cv=cross_val_score(estimator=xg,X=xtrain,y=ytrain,cv=10)
print(abs(cv.mean()))


In [ ]:
acc5=accuracy_score(ytest,ypred5)
acc5

In [ ]:
cm5=confusion_matrix(ytest,ypred5)
plt.figure(figsize=(10,7))
plt.title('Confusion matrix for XGBoost')
df_cm=pd.DataFrame(cm5,index=[i for i in [0,1]],columns=[i for i in ['predicted 0','predicted 1']])
sns.heatmap(df_cm,annot=True,fmt='g');

In [ ]:
algo=['RandomForest','Kneighbors','Naive_Bayes','DecisionTree','Logistic','XGBoost']
accuracy=[acc*100,acc1*100,acc2*100,acc3*100,acc4*100,acc5*100]
results=pd.DataFrame({'algorithm':algo,'accuracy(%)':accuracy})
results=results[['algorithm','accuracy(%)']]
results

so, the RandomForest & XGBoost classifiers have performed well on the given dataset.